In [1]:
from google.cloud import storage
import scipy
from scipy import signal
from scipy.signal import find_peaks
from scipy import stats
from scipy.stats import shapiro
import numpy as np
import pandas as pd
import os
import sys
import scipy.interpolate as interp
sys.path.append(os.path.join(sys.path[0], '..', 'src'))
import excel_reader_gcp as excel_reader
import excel_reader_gcp_GN as excel_reader_GN
print("The module is located at:", excel_reader.__file__)
import datetime
import logging   
import csv
import matplotlib.pyplot as plt
from hmmlearn import hmm
from sklearn.model_selection import train_test_split
import random
import copy
import re
import gait_metrics as gait_metrics
from gait_metrics import *



def reshape_vector(vectors_orig, new_size=40, num_axes=3):
    x_new = np.linspace(0, 100, new_size)
    trial_reshaped = []
    for stride in vectors_orig:
        x_orig = np.linspace(0, 100, len(stride))
        func_cubic = [interp.interp1d(x_orig, stride[:, i], kind='cubic') for i in range(num_axes)]
        vec_cubic = np.array([func_cubic[i](x_new) for i in range(num_axes)]).transpose()
        trial_reshaped.append(vec_cubic)
    return np.array(trial_reshaped)

The module is located at: c:\GP-WearablesAnalysis\examples\excel_reader_gcp.py


In [7]:
def finding_groupings(num_groups, gait_parameter, gait_cycles, part_strides, spatiotemp, percent_grading, reverse=True):

    # (Same code as before until we need to handle part_strides)
    if reverse:
        percent_grading = -percent_grading
        values_sorted = sorted(gait_parameter, reverse=True)
        sorted_indices = np.argsort(gait_parameter)[::-1]  # Sort indices in descending order of stance time symmetry
    else:
        sorted_indices = np.argsort(gait_parameter)
        values_sorted = sorted(gait_parameter, reverse=False)

    n = len(gait_parameter)
    group_sizes = [n // num_groups + (1 if i < n % num_groups else 0) for i in range(num_groups)]
    target_means = [np.mean(values_sorted[:group_sizes[0]])]

    # Initializes the groups and the remaining values to be picked from
    groups = [[] for _ in range(num_groups)]
    grouped_gait_cycles = [[] for _ in range(num_groups)]
    remaining_indices = sorted_indices[:]

    for i in range(1, num_groups):
        target_means.append(target_means[0] + percent_grading * i)

    # Initializes the groups and the remaining values to be picked from
    groups = [[] for _ in range(num_groups)]
    grouped_gait_cycles = [[] for _ in range(num_groups)]
    grouped_strides_dict = [{} for _ in range(num_groups)]
    grouped_spatiotemp_dict = [{} for _ in range(num_groups)]
     
    remaining_indices = sorted_indices[:]

    for i in range(1, num_groups):
        target_means.append(target_means[0] + percent_grading * i)

    params = spatiotemp[trial_type]
    
    for i in range(num_groups):
        target_mean = target_means[i]
        filtered_indices = [idx for idx in remaining_indices if abs(gait_parameter[idx] - target_mean) < percent_grading / 2]
        selected_indices = filtered_indices[:]
        
        groups[i].extend(gait_parameter[idx] for idx in selected_indices)
        grouped_gait_cycles[i].extend(gait_cycles[idx] for idx in selected_indices)
        
        for key, data in part_strides[trial_type].items():
            # Extract the desired indices from each array
            selected_data = []
            for array in data:
                # Check if array is long enough
                if len(array) > max(selected_indices):
                    selected_values = [array[i] for i in selected_indices if i < len(array)]
                    selected_data.append(selected_values)
            grouped_strides_dict[i][key] = selected_data
            
        subsampled_params = []
        for param in params:
            subsampled_param = param[:, selected_indices]
            subsampled_params.append(subsampled_param)
    
        # Add the subsampled arrays to the new dictionary
        grouped_spatiotemp_dict[i] = subsampled_params
        
        remaining_indices = [idx for idx in remaining_indices if idx not in selected_indices]
            
    return groups, grouped_gait_cycles, grouped_strides_dict, grouped_spatiotemp_dict

def random_sampling(groups, grouped_gait_cycles, grouped_strides_dict, grouped_spatiotemp_dict, sample_size=50):
    def adaptive_subsample(group, first_mean, i, percent_grading=0.03, tolerance=0.005, sample_size=50, max_iterations=10000):
        available_indices = list(range(len(group)))  # List of all indices in the group
        sample_indices = np.random.choice(available_indices, size=sample_size, replace=False)
        
        for idx in sample_indices:
            available_indices.remove(idx)  # Remove initial sample values from available values
        
        for _ in range(max_iterations):
            current_mean = np.mean([group[idx] for idx in sample_indices])
            percent_diff = current_mean - first_mean 
            target_diff = percent_grading * i

            if len(available_indices) == 0:
                raise ValueError("No candidates available to adjust the mean")
            
            if (target_diff - tolerance) <= abs(percent_diff) <= (target_diff + tolerance):
                return sample_indices
            
            if abs(percent_diff) < (target_diff - tolerance):
                if percent_diff < 0:
                    # Choose a new sample from the lower half
                    lower_idx = [idx for idx in available_indices if group[idx] <= np.percentile(group, 50)]
                    if lower_idx:
                        new_idx = np.random.choice(lower_idx)
                        sample_indices = np.append(sample_indices, new_idx)
                    else:
                        new_idx = np.random.choice(available_indices)
                        sample_indices = np.append(sample_indices, new_idx)
                    
                    available_indices.remove(new_idx)
                    sample_indices = np.delete(sample_indices, np.argmax([group[idx] for idx in sample_indices]))
                else:
                    # Choose a new sample from the upper half
                    higher_idx = [idx for idx in available_indices if group[idx] >= np.percentile(group, 50)]
                    if higher_idx:
                        new_idx = np.random.choice(higher_idx)
                        sample_indices = np.append(sample_indices, new_idx)
                    else:
                        new_idx = np.random.choice(available_indices)
                        sample_indices = np.append(sample_indices, new_idx)
                    
                    available_indices.remove(new_idx)
                    sample_indices = np.delete(sample_indices, np.argmin([group[idx] for idx in sample_indices]))
            else:
                if percent_diff > 0:
                    lower_idx = [idx for idx in available_indices if group[idx] <= np.percentile(group, 50)]
                    if lower_idx:
                        new_idx = np.random.choice(lower_idx)
                        sample_indices = np.append(sample_indices, new_idx)
                    else:
                        new_idx = np.random.choice(available_indices)
                        sample_indices = np.append(sample_indices, new_idx)
                    
                    available_indices.remove(new_idx)
                    sample_indices = np.delete(sample_indices, np.argmax([group[idx] for idx in sample_indices]))
                else:
                    # Choose a new sample from the upper half
                    higher_idx = [idx for idx in available_indices if group[idx] >= np.percentile(group, 50)]
                    if higher_idx:
                        new_idx = np.random.choice(higher_idx)
                        sample_indices = np.append(sample_indices, new_idx)
                    else:
                        new_idx = np.random.choice(available_indices)
                        sample_indices = np.append(sample_indices, new_idx)
                    
                    available_indices.remove(new_idx)
                    sample_indices = np.delete(sample_indices, np.argmin([group[idx] for idx in sample_indices]))

        raise ValueError("Could not find suitable subsample within the maximum number of iterations")

    groups_subsampled_list = []
    gaitcycles_subsampled_list = []
    
    # Sample for the first group
    indices_first_group = list(range(len(groups[0])))
    sample_indices_first_group = np.random.choice(indices_first_group, size=sample_size * 2, replace=False)
    group1_mean = np.mean([groups[0][idx] for idx in sample_indices_first_group])  # First mean used as the target for all subsequent groups

    random.shuffle(sample_indices_first_group)
    
    # ###REMOVE LATER
    
    grouped_strides_dict_subsampled = [{} for _ in range(4)]
    grouped_spatiotemp_dict_subsampled = [{} for _ in range(4)]
    
    # for key, data in grouped_strides_dict[0].items():
    #     selected_data = []
    #     for array in data:
    #         # Check if array is long enough
    #         if len(array) > max(sample_indices_first_group):
    #             selected_values = [array[i] for i in sample_indices_first_group if i < len(array)]
    #             selected_data.append(selected_values)
    #     grouped_strides_dict_subsampled[0][key] = selected_data
    
    # params = grouped_spatiotemp_dict[0]
    
    # subsampled_params = []
    # for param in params:
    #     subsampled_param = param[:, sample_indices_first_group]
    #     subsampled_params.append(subsampled_param)
    # # Add the subsampled arrays to the new dictionary
    # grouped_spatiotemp_dict_subsampled[0] = subsampled_params    
        
    # subsampled_values_FULLBASELINE = [groups[0][j] for j in sample_indices_first_group]
    # subsampled_gait_cycles_FULLBASELINE = [grouped_gait_cycles[0][j] for j in sample_indices_first_group]
    
    # groups_subsampled_list.append(subsampled_values_FULLBASELINE)
    # gaitcycles_subsampled_list.append(subsampled_gait_cycles_FULLBASELINE)
    

    baseline_1_indices = sample_indices_first_group[:50]
    baseline_2_indices = sample_indices_first_group[50:]

    subsampled_values_baseline1 = [groups[0][j] for j in baseline_1_indices]
    subsampled_gait_cycles_baseline1 = [grouped_gait_cycles[0][j] for j in baseline_1_indices]

    subsampled_values_baseline2 = [groups[0][j] for j in baseline_2_indices]
    subsampled_gait_cycles_baseline2 = [grouped_gait_cycles[0][j] for j in baseline_2_indices]

    groups_subsampled_list.append(subsampled_values_baseline1)
    groups_subsampled_list.append(subsampled_values_baseline2)

    gaitcycles_subsampled_list.append(subsampled_gait_cycles_baseline1)
    gaitcycles_subsampled_list.append(subsampled_gait_cycles_baseline2)

    
    for key, data in grouped_strides_dict[0].items():
        selected_data = []
        for array in data:
            # Check if array is long enough
            if len(array) > max(baseline_1_indices):
                selected_values = [array[i] for i in baseline_1_indices if i < len(array)]
                selected_data.append(selected_values)
        grouped_strides_dict_subsampled[0][key] = selected_data
    
    for key, data in grouped_strides_dict[0].items():
        selected_data = []
        for array in data:
            # Check if array is long enough
            if len(array) > max(baseline_2_indices):
                selected_values = [array[i] for i in baseline_2_indices if i < len(array)]
                selected_data.append(selected_values)
        grouped_strides_dict_subsampled[1][key] = selected_data  
        
    params = grouped_spatiotemp_dict[0]
    
    subsampled_params = []
    for param in params:
        subsampled_param = param[:, baseline_1_indices]
        subsampled_params.append(subsampled_param)
    # Add the subsampled arrays to the new dictionary
    grouped_spatiotemp_dict_subsampled[0] = subsampled_params
    
    subsampled_params = []
    for param in params:
        subsampled_param = param[:, baseline_2_indices]
        subsampled_params.append(subsampled_param)
    # Add the subsampled arrays to the new dictionary
    grouped_spatiotemp_dict_subsampled[1] = subsampled_params
    
    # Only consider 3 groups here
    for i in range(1, 3):
        sample_indices = adaptive_subsample(np.array(groups[i]), group1_mean, i)
        subsampled_values = [groups[i][j] for j in sample_indices]
        subsampled_gait_cycles = [grouped_gait_cycles[i][j] for j in sample_indices]
        groups_subsampled_list.append(subsampled_values)
        gaitcycles_subsampled_list.append(subsampled_gait_cycles)
        
        for key, data in grouped_strides_dict[i].items():
            selected_data = []
            for array in data:
                # Check if array is long enough
                if len(array) > max(sample_indices):
                    selected_values = [array[i] for i in sample_indices if i < len(array)]
                    selected_data.append(selected_values)
            
            grouped_strides_dict_subsampled[i+1][key] = selected_data  
        
        params = grouped_spatiotemp_dict[i]    
        subsampled_params = []
        for param in params:
            subsampled_param = param[:, sample_indices]
            subsampled_params.append(subsampled_param)
        # Add the subsampled arrays to the new dictionary
        grouped_spatiotemp_dict_subsampled[i+1] = subsampled_params
                
    return groups_subsampled_list, gaitcycles_subsampled_list, grouped_strides_dict_subsampled, grouped_spatiotemp_dict_subsampled


def check_group_configurations(gait_split_parameter, raw_sensor_data, part_strides, spatiotemp):
    percent_grading = 0.03
    groups, grouped_gait_cycles, grouped_strides_dict, grouped_spatiotemp_dict = finding_groupings(4, gait_split_parameter, raw_sensor_data, part_strides, spatiotemp, percent_grading, reverse=False)
    
    filtered_groups = []
    filtered_gait_groups = []
    filtered_strides = []
    filtered_spatiotemp = []
    
    for i in range(len(groups)):
        if len(groups[i]) > 70:
            filtered_groups.append(groups[i])
            filtered_gait_groups.append(grouped_gait_cycles[i])
            filtered_strides.append(grouped_strides_dict[i])
            filtered_spatiotemp.append(grouped_spatiotemp_dict[i])
    
    if len(filtered_groups) < 3:
        groups, grouped_gait_cycles, grouped_strides_dict, grouped_spatiotemp_dict = finding_groupings(4, gait_split_parameter, raw_sensor_data, part_strides, spatiotemp, percent_grading, reverse=True)  # Try the other direction if requirements are not fulfilled
        filtered_groups = []
        filtered_gait_groups = []
        filtered_strides = []
        filtered_spatiotemp = []
        
        for i in range(len(groups)):
            if len(groups[i]) > 70:
                filtered_groups.append(groups[i])
                filtered_gait_groups.append(grouped_gait_cycles[i])
                filtered_strides.append(grouped_strides_dict[i])
                filtered_spatiotemp.append(grouped_spatiotemp_dict[i])
    
        if len(filtered_groups) < 3:
            raise ValueError("Insufficient group sizes available for this participant")
    
    groups, gaitcycles, strides, spatiotemp = random_sampling(filtered_groups, filtered_gait_groups, filtered_strides, filtered_spatiotemp)
    
    return groups, gaitcycles, strides, spatiotemp

In [8]:
#Establishing control for GPS

storage_client = storage.Client()
part_strides = {}
part_gait_params = {}
part_kinematic_params = {}
control_strides = {}
control_gait_params = {}
control_kinematic_params = {}


bucket_dir = 'gs://gaitbfb_propellab/'


def reshape_vector(vectors_orig, new_size=40, num_axes=3):
    x_new = np.linspace(0, 100, new_size)
    trial_reshaped = []
    for stride in vectors_orig:
        x_orig = np.linspace(0, 100, len(stride))
        func_cubic = [interp.interp1d(x_orig, stride[:, i], kind='cubic') for i in range(num_axes)]
        vec_cubic = np.array([func_cubic[i](x_new) for i in range(num_axes)]).transpose()
        trial_reshaped.append(vec_cubic)
    return np.array(trial_reshaped)

def compile_gait_data(store_gait_cycles, store_gait_params, store_kin_params, filenames, trial_type_filter, print_filenames=False, look_at_all_files = True, desired_filetypes=None):   

    XsensGaitParser = excel_reader_GN.XsensGaitDataParser()  
    for i, file in enumerate(sorted(filenames)):
        trial_type = re.search(trial_type_filter, file).group(1)
        if(look_at_all_files or any(filetype in file for filetype in desired_filetypes)):
            XsensGaitParser.process_mvn_trial_data(os.path.join(bucket_dir, file))
            partitioned_mvn_data = XsensGaitParser.get_partitioned_mvn_data()
            gait_params = XsensGaitParser.get_gait_param_info()

            if trial_type in store_gait_cycles:
                for body_part in store_gait_cycles[trial_type]:
                    for i, side in enumerate(store_gait_cycles[trial_type][body_part]):
                        # for each part (pelvis, l_hip, r_knee, etc.), append strides to appropriate list
                        store_gait_cycles[trial_type][body_part][i] = store_gait_cycles[trial_type][body_part][i] + partitioned_mvn_data[body_part][i]

                store_gait_params[trial_type].append(gait_params['spatio_temp'])

                for joint in store_kin_params[trial_type]:
                    for i, side in enumerate(store_kin_params[trial_type][joint]):
                        store_kin_params[trial_type][joint][i] = np.append(store_kin_params[trial_type][joint][i], gait_params['kinematics'][joint][i], axis=0)

            else:
                store_gait_cycles[trial_type] = partitioned_mvn_data
                store_gait_params[trial_type] = [gait_params['spatio_temp']]
                store_kin_params[trial_type] = gait_params['kinematics']


bucket_name = 'gaitbfb_propellab/'
base_directory = bucket_name + 'Wearable Biofeedback System (REB-0448)/Data/Raw Data'
bucket_name = 'gaitbfb_propellab'
prefix = 'control_dir'
control_dir = 'Gait Quality Analysis/Data/Participant_Data/Processed Data/AbleBodied_Control/CSV'
blobs = storage_client.list_blobs(bucket_name, prefix = control_dir)
control_files = []
for blob in blobs:
    if('.csv' in blob.name):
        control_files.append(blob.name)

compile_gait_data(control_strides, control_gait_params, control_kinematic_params, control_files, 'CSV/(.*?)-00')        

aggregate_control_data = {}
strides_per_control = 10
for i, indiv in enumerate(control_strides.keys()):
    indices = np.arange(len(control_strides[indiv]['gyro_data'][0]))
    np.random.shuffle(indices)
    #control_strides_per_part.append(min(strides_per_control, len(indices)))
    
    if(i == 0):
        aggregate_control_data = control_strides[indiv]
        
        for signal_type in control_strides[indiv]:
            for j, side in enumerate(control_strides[indiv][signal_type]):
                aggregate_control_data[signal_type][j] = [control_strides[indiv][signal_type][j][indices[k]] for k in range(min(strides_per_control, len(indices))) ]
                                    
    else:
        # randomly sample 10 gait cycles from each able-bodied in control, or all gait cycles if less than 10
        for signal_type in control_strides[indiv]:
            for j, side in enumerate(control_strides[indiv][signal_type]):
                aggregate_control_data[signal_type][j] = aggregate_control_data[signal_type][j] + [control_strides[indiv][signal_type][j][indices[k]] 
                                                                                                for k in range(min(strides_per_control, len(indices))) ]

# reshape all the kinematic signals to the size specified for the GPS (51, e.g. 2% increments across the gait cycles from HS to HS)
# store in partitioned_awinda_control
partitioned_awinda_control = {}
partitioned_awinda_control['pelvis_orient'] = reshape_vector(aggregate_control_data['pelvis_orient'][0], new_size = 51)
partitioned_awinda_control['hip_angle'] = [reshape_vector(aggregate_control_data['hip_angle'][0], new_size = 51), reshape_vector(aggregate_control_data['hip_angle'][1], new_size = 51)]
partitioned_awinda_control['knee_angle'] = [reshape_vector(aggregate_control_data['knee_angle'][0], new_size = 51), reshape_vector(aggregate_control_data['knee_angle'][1], new_size = 51)]
partitioned_awinda_control['ankle_angle'] = [reshape_vector(aggregate_control_data['ankle_angle'][0], new_size = 51), reshape_vector(aggregate_control_data['ankle_angle'][1], new_size = 51)]

c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=

In [9]:
### USING AB AS CONTROL####

XsensGaitParser =  excel_reader_GN.XsensGaitDataParser()
storage_client = storage.Client()
part_strides = {}
part_gait_params = {}
part_kinematic_params = {}
control_strides = {}
control_gait_params = {}
control_kinematic_params = {}

params_INI = {
            'stride time':[],
            'stride length':[],
            'swing phase':[],
            'MAV':[],
            'MAH':[],
            'MHD':[],
            'MAB':[],
            'MAD':[],
            'ShROM':[]
             }

ini_height_norm = {
            'stride time':False,
            'stride length':True,
            'swing phase':False,
            'MAV':True,
            'MAH':True,
            'MHD':True,
            'MAB':True,
            'MAD':True,
            'ShROM':False
            }
params_temporal_mgs = {
            'stance time':[],
            'swing time':[],
            'double support':[],
            'step time':[],
            'stride time':[]
            }

bucket_dir = 'gs://gaitbfb_propellab/'
def compile_gait_data(store_gait_cycles, store_gait_params, store_kin_params, filenames, trial_type_filter, print_filenames=False, look_at_all_files = True, desired_filetypes=None):   
 
    XsensGaitParser = excel_reader_GN.XsensGaitDataParser()  
    for i, file in enumerate(sorted(filenames)):
        trial_type = re.search(trial_type_filter, file).group(1)

        if(look_at_all_files or any(filetype in file for filetype in desired_filetypes)):
            XsensGaitParser.process_mvn_trial_data(os.path.join(bucket_dir, file))
            partitioned_mvn_data = XsensGaitParser.get_partitioned_mvn_data()
            gait_params = XsensGaitParser.get_gait_param_info()

            if trial_type in store_gait_cycles:
                for body_part in store_gait_cycles[trial_type]:
                    for j, side in enumerate(store_gait_cycles[trial_type][body_part]):
                        # for each part (pelvis, l_hip, r_knee, etc.), append strides to appropriate list
                        store_gait_cycles[trial_type][body_part][j] = store_gait_cycles[trial_type][body_part][j] + partitioned_mvn_data[body_part][j]

                for j, param in enumerate(store_gait_params[trial_type]):
                    store_gait_params[trial_type][j] = np.concatenate( (store_gait_params[trial_type][j], gait_params['spatio_temp'][j]), axis=-1)

                for joint in store_kin_params[trial_type]:
                    for j, side in enumerate(store_kin_params[trial_type][joint]):
                        store_kin_params[trial_type][joint][j] = np.append(store_kin_params[trial_type][joint][j], gait_params['kinematics'][joint][j], axis=0)

            else:
                store_gait_cycles[trial_type] = partitioned_mvn_data
                store_gait_params[trial_type] = gait_params['spatio_temp']
                store_kin_params[trial_type] = gait_params['kinematics']
                
                
bucket_name = 'gaitbfb_propellab/'
base_directory = bucket_name + 'Wearable Biofeedback System (REB-0448)/Data/Raw Data'
bucket_name = 'gaitbfb_propellab'
prefix = 'control_dir'
control_dir = 'Gait Quality Analysis/Data/Participant_Data/Processed Data/AbleBodied_Control/CSV'
blobs = storage_client.list_blobs(bucket_name, prefix = control_dir)
control_files = []
for blob in blobs:
    if('.csv' in blob.name):
        control_files.append(blob.name)
participant_info = pd.read_csv(bucket_dir + 'Gait Quality Analysis/Data/Participant_Data/Raw Data/participant_info.csv')
compile_gait_data(control_strides, control_gait_params, control_kinematic_params, control_files, 'CSV/(.*?)-00')        

aggregate_control_data = {}
strides_per_control = 10
control_strides_per_part = []
for i, indiv in enumerate(control_strides.keys()):
    
    indiv_height = participant_info.loc[participant_info['Participant'] == indiv]['Height (m)'].item()
    for j, param in enumerate(params_INI):
        if(ini_height_norm[param]):
            control_gait_params[indiv][j] = control_gait_params[indiv][j] / indiv_height  

    indices = np.arange(len(control_strides[indiv]['gyro_data'][0]))
    np.random.shuffle(indices)
    control_strides_per_part.append(min(strides_per_control, len(indices)))
    
    if(i == 0):
        aggregate_control_data = control_strides[indiv]
        
        for signal_type in control_strides[indiv]:
            for j, side in enumerate(control_strides[indiv][signal_type]):
                aggregate_control_data[signal_type][j] = [control_strides[indiv][signal_type][j][indices[k]] for k in range(min(strides_per_control, len(indices))) ]
                    
        for j, param in enumerate(params_INI.keys()):
            for k in range(2):
                params_INI[param].append([])
                params_INI[param][k] = [control_gait_params[indiv][j][k][indices[a]] for a in range(min(strides_per_control, len(indices))) ]
        
    else:
        # choose 10 gait cycles randomly from each able-bodied participant, or all gait cycles if less than 10
        for signal_type in control_strides[indiv]:
            for j, side in enumerate(control_strides[indiv][signal_type]):
                aggregate_control_data[signal_type][j] = aggregate_control_data[signal_type][j] + [control_strides[indiv][signal_type][j][indices[k]] 
                                                                                                   for k in range(min(strides_per_control, len(indices))) ]
        for j, param in enumerate(params_INI.keys()):
            for k in range(2):
                params_INI[param][k] = params_INI[param][k] + [control_gait_params[indiv][j][k][indices[a]] for a in range(min(strides_per_control, len(indices))) ]
                

print('Parsed control data...')
for param in params_INI:
    params_INI[param] = np.array(params_INI[param])
    
class AwindaData():
    def __init__(self, trial_type, partitioned_movement_data, ini_gait_params = None, mgs_temporal_params = None):
        self.trial_type = trial_type
        self.partitioned_movement_data = partitioned_movement_data
        self.ini_gait_params = ini_gait_params

def combine_acc_and_gyro(numpy_acc, numpy_gyro):
    return np.concatenate((numpy_acc, numpy_gyro), axis=-1)

acc_scale = 0.02
partitioned_awinda_control = {}
partitioned_awinda_control['Pelvis_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(aggregate_control_data['acc_data'][1]), reshape_vector(aggregate_control_data['gyro_data'][1]))
partitioned_awinda_control['UpperR_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(aggregate_control_data['acc_data'][2]), reshape_vector(aggregate_control_data['gyro_data'][2]))
partitioned_awinda_control['UpperL_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(aggregate_control_data['acc_data'][5]), reshape_vector(aggregate_control_data['gyro_data'][5]))
partitioned_awinda_control['LowerR_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(aggregate_control_data['acc_data'][3]), reshape_vector(aggregate_control_data['gyro_data'][3]))
partitioned_awinda_control['LowerL_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(aggregate_control_data['acc_data'][6]), reshape_vector(aggregate_control_data['gyro_data'][6]))

partitioned_awinda_control['pelvis_orient'] = reshape_vector(aggregate_control_data['pelvis_orient'][0], new_size = 51)
partitioned_awinda_control['hip_angle'] = [reshape_vector(aggregate_control_data['hip_angle'][0], new_size = 51), reshape_vector(aggregate_control_data['hip_angle'][1], new_size = 51)]
partitioned_awinda_control['knee_angle'] = [reshape_vector(aggregate_control_data['knee_angle'][0], new_size = 51), reshape_vector(aggregate_control_data['knee_angle'][1], new_size = 51)]
partitioned_awinda_control['ankle_angle'] = [reshape_vector(aggregate_control_data['ankle_angle'][0], new_size = 51), reshape_vector(aggregate_control_data['ankle_angle'][1], new_size = 51)]

partitioned_control_data = AwindaData('control', dict(partitioned_awinda_control), dict(params_INI))

c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=

Parsed control data...


In [12]:
#Using AB as control

script_dir = 'C:\GP-WearablesAnalysis\examples'
run_time = datetime.datetime.now().strftime("%d-%m-%y_%H-%M")
csv_filename = f"logresults_STSR_INI_ABControl_{run_time}.csv" #Builds a log file based on the current time to keep track of runs
csv_path = os.path.join(script_dir, csv_filename)

def add_row_to_csv(csv_path, gait_param, algorithm, participant_num, level, parameter):
    with open(csv_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([csv_path, gait_param, algorithm, participant_num, level, parameter])

##### AGGREGATE TEST DATA AND EVAL INI #####

XsensGaitParser =  excel_reader.XsensGaitDataParser() #Different excel reader file, which contains the spatiotemporal parameters not pertaining to the INI - used to be able to get stance time symmetry
storage_client = storage.Client()
bucket_name = 'gaitbfb_propellab/'
base_directory = bucket_name + 'Wearable Biofeedback System (REB-0448)/Data/Raw Data'
bucket_name = 'gaitbfb_propellab'
blobs = storage_client.list_blobs(bucket_name, prefix = base_directory)
prefix_from_bucket = 'Wearable Biofeedback System (REB-0448)/Data/Raw Data/' 

participant_list = ['LLPU_P01','LLPU_P02','LLPU_P03','LLPU_P04','LLPU_P05','LLPU_P06','LLPU_P09','LLPU_P10','LLPU_P12','LLPU_P15']
arrangements = ['pelvis','upper','lower']

def organize_signals(sensor_mappings, gyro_signal, accel_signal):
    combined_signals = {}
    for location, sensor in sensor_mappings.items():
        reshaped_gyro = reshape_vector(gyro_signal[sensor], 40, 3)
        reshaped_accel = reshape_vector(accel_signal[sensor], 40, 3)
        combined_signals[location] = np.concatenate((reshaped_gyro, reshaped_accel), axis=2) #Concatenates to gyro x,y,z and accel x,y,z
    return combined_signals

def calc_INI(control_mean, control_std, eig_vals, eig_vecs, data):
    standardized_data = (data - control_mean) / control_std
    transformed_data = (np.dot(standardized_data, eig_vecs)) / np.sqrt(eig_vals)
    
    return transformed_data

#Dictionary to map the sensor locations to their IDs.
sensor_mappings = {
    'pelvis': 1,
    'UpperR': 2,
    'LowerR': 3,
    'UpperL': 5,
    'LowerL': 6
}
     
for participant in participant_list:
    print(f"Processing participant {participant}")
 
    directory = prefix_from_bucket + participant + '/Excel_Data_Trimmed'
    blobs = storage_client.list_blobs(bucket_or_name=bucket_name, prefix=directory.replace("\\", "/"))
    part_strides = {}
    part_gait_params = {}
    part_kinematic_params = {}
    part_raw_sensor = []
    trial_type = 'LLPU'
    height_normalized = True
    
    part_gait_params_INI = {}
    part_strides_INI = {}
   
    logging.info(f"Processing participant {participant}")
    if blobs:
        for blob in blobs:
            if blob.name.endswith('.csv'):
                try:
                    XsensGaitParser.process_mvn_trial_data(f"gs://{bucket_name}/{blob.name}")
                    partitioned_mvn_data = XsensGaitParser.get_partitioned_mvn_data()
                    gait_params = XsensGaitParser.get_gait_param_info()
                    combined_signals = organize_signals(sensor_mappings, partitioned_mvn_data['gyro_data'], partitioned_mvn_data['acc_data'])
                    pelvis_data = combined_signals['pelvis']
                    upper_data = np.concatenate((combined_signals['UpperR'], combined_signals['UpperL']), axis=2)  # Concatenate by last axis
                    lower_data = np.concatenate((combined_signals['LowerR'], combined_signals['LowerL']), axis=2)  # Concatenate by last axis
                    full_sensors = np.concatenate((pelvis_data,upper_data,lower_data),axis=2)
                    part_raw_sensor.append(full_sensors)

                    if trial_type in part_strides:
                        for body_part in part_strides[trial_type]:
                            for i, side in enumerate(part_strides[trial_type][body_part]):
                                # for each part (pelvis, l_hip, r_knee, etc.), append strides to appropriate list
                                part_strides[trial_type][body_part][i] = part_strides[trial_type][body_part][i] + partitioned_mvn_data[body_part][i]
                        
                        part_gait_params[trial_type].append(gait_params['spatio_temp'])
                        
                        for joint in part_kinematic_params[trial_type]:
                            for i, side in enumerate(part_kinematic_params[trial_type][joint]):
                                part_kinematic_params[trial_type][joint][i] = np.append(part_kinematic_params[trial_type][joint][i], gait_params['kinematics'][joint][i], axis=0) 

                    else:
                        part_strides[trial_type] = partitioned_mvn_data
                        part_kinematic_params[trial_type] = gait_params['kinematics']
                        part_gait_params[trial_type] = [gait_params['spatio_temp']]
                    
                    file_name = os.path.basename(blob.name)
                    
                    ##CHANGE THE excel reader - access the different 
                    XsensGaitParser_GN =  excel_reader_GN.XsensGaitDataParser()
                    XsensGaitParser_GN.process_mvn_trial_data(f"gs://{bucket_name}/{blob.name}")
                    partitioned_mvn_data_GN = XsensGaitParser_GN.get_partitioned_mvn_data()
                    gait_params_GN = XsensGaitParser_GN.get_gait_param_info()
                    
                    if trial_type in part_strides_INI:
                        for body_part in part_strides[trial_type]:
                            for i, side in enumerate(part_strides_INI[trial_type][body_part]):
                                # for each part (pelvis, l_hip, r_knee, etc.), append strides to appropriate list
                                part_strides_INI[trial_type][body_part][i] = part_strides_INI[trial_type][body_part][i] + partitioned_mvn_data_GN[body_part][i]
                        
                        for j, param in enumerate(part_gait_params_INI[trial_type]):
                            part_gait_params_INI[trial_type][j] = np.concatenate((part_gait_params_INI[trial_type][j], gait_params_GN['spatio_temp'][j]), axis=-1)

                    else:
                        part_strides_INI[trial_type] = partitioned_mvn_data_GN
                        part_gait_params_INI[trial_type] = gait_params_GN['spatio_temp']
                    

                except IndexError as e: #Exception based on an Index Error encountered in excel_reader_gcp.py **
                    #print(f"File skipped: gs://{bucket_name}/{blob.name} due to error: {e}")
                    continue                              
    
    if trial_type in part_gait_params:
        
        stance_time_symmetry = [item for sublist in [i[11] for i in part_gait_params[trial_type]] for item in sublist]
        
        flattened_raw_sensor = []
        part_strides_list = []
        for sublist in part_raw_sensor:
            for item in sublist:
                flattened_raw_sensor.append(item) #Flatten to individual gait cycles
        
        partitioned_awinda_gait = {}
        partitioned_awinda_gait['pelvis_orient'] = reshape_vector(part_strides[trial_type]['pelvis_orient'][0], new_size = 51)
        partitioned_awinda_gait['hip_angle'] = [reshape_vector(part_strides[trial_type]['hip_angle'][0], new_size = 51), reshape_vector(part_strides[trial_type]['hip_angle'][1], new_size = 51)]
        partitioned_awinda_gait['knee_angle'] = [reshape_vector(part_strides[trial_type]['knee_angle'][0], new_size = 51), reshape_vector(part_strides[trial_type]['knee_angle'][1], new_size = 51)]
        partitioned_awinda_gait['ankle_angle'] = [reshape_vector(part_strides[trial_type]['ankle_angle'][0], new_size = 51), reshape_vector(part_strides[trial_type]['ankle_angle'][1], new_size = 51)]


        # Extract and reshape individual signals
        individual_signals = []
        gait_scores_list = []


        for i in range(partitioned_awinda_gait['pelvis_orient'].shape[0]):
            signal_dict = {
                'pelvis_orient': partitioned_awinda_gait['pelvis_orient'][i].reshape(1, 51, 3),
                'hip_angle': [partitioned_awinda_gait['hip_angle'][0][i].reshape(1, 51, 3), partitioned_awinda_gait['hip_angle'][1][i].reshape(1, 51, 3)],
                'knee_angle': [partitioned_awinda_gait['knee_angle'][0][i].reshape(1, 51, 3), partitioned_awinda_gait['knee_angle'][1][i].reshape(1, 51, 3)],
                'ankle_angle': [partitioned_awinda_gait['ankle_angle'][0][i].reshape(1, 51, 3), partitioned_awinda_gait['ankle_angle'][1][i].reshape(1, 51, 3)]
            }
            individual_signals.append(signal_dict)

        print(np.shape(individual_signals[0]['pelvis_orient']))    
        print(np.shape(individual_signals[0]['hip_angle'][0]))

        for signal_val in individual_signals:
            gait_scores = calc_gait_profile_score(signal_val, partitioned_awinda_control)
            gait_scores_list.append(gait_scores)
                
        groups, gaitcycles, ordered_strides, ordered_params = check_group_configurations(stance_time_symmetry, flattened_raw_sensor, part_strides, part_gait_params_INI)
        
        #AB AS CONTROL FOR INI
        
        participant_info = pd.read_csv(bucket_dir + 'Gait Quality Analysis/Data/Participant_Data/Raw Data/participant_info.csv')
        partitioned_gait_data = []
        
        for index, part_strides in enumerate(ordered_strides):
    
            params_INI = {
                    'stride time':[],
                    'stride length':[],
                    'swing phase':[],
                    'MAV':[],
                    'MAH':[],
                    'MHD':[],
                    'MAB':[],
                    'MAD':[],
                    'ShROM':[]
                    }
            
            acc_scale = 0.02
            partitioned_signals_awinda_grouped = {}
            partitioned_signals_awinda_grouped['Pelvis_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(part_strides['acc_data'][1]), reshape_vector(part_strides['gyro_data'][1]))
            partitioned_signals_awinda_grouped['UpperR_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(part_strides['acc_data'][2]), reshape_vector(part_strides['gyro_data'][2]))
            partitioned_signals_awinda_grouped['UpperL_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(part_strides['acc_data'][5]), reshape_vector(part_strides['gyro_data'][5]))
            partitioned_signals_awinda_grouped['LowerR_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(part_strides['acc_data'][3]), reshape_vector(part_strides['gyro_data'][3]))
            partitioned_signals_awinda_grouped['LowerL_IMU'] = combine_acc_and_gyro(acc_scale*reshape_vector(part_strides['acc_data'][6]), reshape_vector(part_strides['gyro_data'][6]))

            partitioned_signals_awinda_grouped['pelvis_orient'] = reshape_vector(part_strides['pelvis_orient'][0], new_size = 51)
            partitioned_signals_awinda_grouped['hip_angle'] = [reshape_vector(part_strides['hip_angle'][0], new_size = 51), 
                                                    reshape_vector(part_strides['hip_angle'][1], new_size = 51)]
            partitioned_signals_awinda_grouped['knee_angle'] = [reshape_vector(part_strides['knee_angle'][0], new_size = 51), 
                                                        reshape_vector(part_strides['knee_angle'][1], new_size = 51)]
            partitioned_signals_awinda_grouped['ankle_angle'] = [reshape_vector(part_strides['ankle_angle'][0], new_size = 51), 
                                                        reshape_vector(part_strides['ankle_angle'][1], new_size = 51)]


            indiv_height = participant_info.loc[participant_info['Participant'] == participant]['Height (m)'].item()
            for i, param in enumerate(params_INI):
                params_INI[param] = ordered_params[index][i]
                if(ini_height_norm[param]):
                    params_INI[param] = params_INI[param] / indiv_height

            partitioned_gait_data.append(AwindaData(trial_type, dict(partitioned_signals_awinda_grouped), dict(params_INI))) 

        control_INI_params = []
        for param in params_INI:
            control_INI_params.append(partitioned_control_data.ini_gait_params[param][0])
        control_INI_params = np.transpose(np.array(control_INI_params))

        mean_control = np.mean(control_INI_params, axis = 0)
        std_control = np.std(control_INI_params, axis = 0)

        standardized_control = (control_INI_params - mean_control) / std_control
        cov_control = np.cov(standardized_control.T)

        eig_vals, eig_vecs = np.linalg.eig(cov_control)
        print(f"INI Score for participant: {participant}")
        for i in range(len(partitioned_gait_data)):
            side_1 = np.transpose(np.array( [partitioned_gait_data[i].ini_gait_params[param][0] for param in params_INI] ))
            side_2 = np.transpose(np.array( [partitioned_gait_data[i].ini_gait_params[param][1] for param in params_INI] ))
            test_ini = np.mean([side_1, side_2], axis=0)
            ini_data = calc_INI(mean_control, std_control, eig_vals, eig_vecs, test_ini)
            ini_score = np.mean(np.linalg.norm(ini_data, axis=-1))
            mean_STSR = np.mean(groups[i])
            add_row_to_csv(csv_path, 'STSR', 'INI', participant, mean_STSR, ini_score)
            #print(mean_STSR)
            print(ini_score)    

<>:3: SyntaxWarning: invalid escape sequence '\G'
<>:3: SyntaxWarning: invalid escape sequence '\G'
C:\Users\ekuep\AppData\Local\Temp\ipykernel_18256\923435260.py:3: SyntaxWarning: invalid escape sequence '\G'
  script_dir = 'C:\GP-WearablesAnalysis\examples'


Processing participant LLPU_P01
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P01
5.873208017055056
5.815506466500247
5.708864165392883
5.698261294251499
Processing participant LLPU_P02
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P02
6.227523637695038
6.217045268685663
5.859681886947804
5.523357262568108
Processing participant LLPU_P03
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P03
4.771827753173785
4.88411772033408
4.753368911571264
4.667817996693739
Processing participant LLPU_P04
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P04
5.614936171129435
5.3792101883981305
5.63325161950306
5.575862334520312
Processing participant LLPU_P05
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P05
3.3574369980912695
3.329748376055621
3.3720692367923073
3.3105897544557905
Processing participant LLPU_P06
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P06
6.041326870751351
5.813686345145352
6.033428616048163
5.907386262864999
Processing participant LLP

In [6]:
#Using first baseline group as control for INI 

script_dir = 'C:\GP-WearablesAnalysis\examples'
run_time = datetime.datetime.now().strftime("%d-%m-%y_%H-%M")
csv_filename = f"logresults_STSR_INI_BaselineControl{run_time}.csv" #Builds a log file based on the current time to keep track of runs
csv_path = os.path.join(script_dir, csv_filename)

def add_row_to_csv(csv_path, gait_param, algorithm, participant_num, level, parameter):
    with open(csv_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([csv_path, gait_param, algorithm, participant_num, level, parameter])


############## PROCESSING INI

XsensGaitParser =  excel_reader.XsensGaitDataParser() #Different excel reader file, which contains the spatiotemporal parameters not pertaining to the INI
storage_client = storage.Client()
bucket_name = 'gaitbfb_propellab/'
base_directory = bucket_name + 'Wearable Biofeedback System (REB-0448)/Data/Raw Data'
bucket_name = 'gaitbfb_propellab'
blobs = storage_client.list_blobs(bucket_name, prefix = base_directory)
prefix_from_bucket = 'Wearable Biofeedback System (REB-0448)/Data/Raw Data/' 

participant_list = ['LLPU_P01','LLPU_P02','LLPU_P03','LLPU_P04','LLPU_P05','LLPU_P06','LLPU_P09','LLPU_P10','LLPU_P12','LLPU_P15']
#participant_list = ['LLPU_P01','LLPU_P02','LLPU_P03','LLPU_P04','LLPU_P05','LLPU_P06','LLPU_P09','LLPU_P10','LLPU_P12','LLPU_P15']
arrangements = ['pelvis','upper','lower']


participant_info_file = pd.read_excel('Q:\\main_propellab\\Users\\Ng, Gabe\\Summer Student 2024\\LLPU_DataSummaries\\LLPU_Height_ProstheticSide.xlsx')

def get_participant_info(participant_id):
    participant = participant_info_file[participant_info_file['Participant_ID'] == participant_id]
    if not participant.empty:
        height = participant.iloc[0]['Height']
        side = int(participant.iloc[0]['Side'])
        return height, side
    else:
        return None, None
    
    
def organize_signals(sensor_mappings, gyro_signal, accel_signal):
    combined_signals = {}
    for location, sensor in sensor_mappings.items():
        reshaped_gyro = reshape_vector(gyro_signal[sensor], 40, 3)
        reshaped_accel = reshape_vector(accel_signal[sensor], 40, 3)
        combined_signals[location] = np.concatenate((reshaped_gyro, reshaped_accel), axis=2) #Concatenates to gyro x,y,z and accel x,y,z
    return combined_signals

def calc_INI(control_mean, control_std, eig_vals, eig_vecs, data):
    standardized_data = (data - control_mean) / control_std
    transformed_data = (np.dot(standardized_data, eig_vecs)) / np.sqrt(eig_vals)
    
    return transformed_data

#Dictionary to map the sensor locations to their IDs.
sensor_mappings = {
    'pelvis': 1,
    'UpperR': 2,
    'LowerR': 3,
    'UpperL': 5,
    'LowerL': 6
}
     
for participant in participant_list:
    print(f"Processing participant {participant}")
    
    participant_id = participant
    height,side = get_participant_info(participant_id)
    height_m = height/100 
    
    if side == 1:
        prosth_side = 0 #Right side
        non_prosth_side = 1
        side_label = "Right"
        print("Prosthetic on the right side")
    else:
        prosth_side = 1 #Left side
        non_prosth_side = 0
        side_label = "Left"
        print("Prosthetic side on the left side")
    
    if height is None or side is None:
        print(f"Participant info not found for ID: {participant_id}")
        continue
    
 
    directory = prefix_from_bucket + participant + '/Excel_Data_Trimmed'
    blobs = storage_client.list_blobs(bucket_or_name=bucket_name, prefix=directory.replace("\\", "/"))
    part_strides = {}
    part_gait_params = {}
    part_kinematic_params = {}
    part_raw_sensor = []
    trial_type = 'LLPU'
    height_normalized = True
    
    part_gait_params_INI = {}
    part_strides_INI = {}
   
    logging.info(f"Processing participant {participant}")
    if blobs:
        for blob in blobs:
            if blob.name.endswith('.csv'):
                try:
                    XsensGaitParser.process_mvn_trial_data(f"gs://{bucket_name}/{blob.name}")
                    partitioned_mvn_data = XsensGaitParser.get_partitioned_mvn_data()
                    gait_params = XsensGaitParser.get_gait_param_info()
                    combined_signals = organize_signals(sensor_mappings, partitioned_mvn_data['gyro_data'], partitioned_mvn_data['acc_data'])
                    pelvis_data = combined_signals['pelvis']
                    upper_data = np.concatenate((combined_signals['UpperR'], combined_signals['UpperL']), axis=2)  # Concatenate by last axis
                    lower_data = np.concatenate((combined_signals['LowerR'], combined_signals['LowerL']), axis=2)  # Concatenate by last axis
                    full_sensors = np.concatenate((pelvis_data,upper_data,lower_data),axis=2)
                    part_raw_sensor.append(full_sensors)

                    if trial_type in part_strides:
                        for body_part in part_strides[trial_type]:
                            for i, side in enumerate(part_strides[trial_type][body_part]):
                                # for each part (pelvis, l_hip, r_knee, etc.), append strides to appropriate list
                                part_strides[trial_type][body_part][i] = part_strides[trial_type][body_part][i] + partitioned_mvn_data[body_part][i]
                        
                        part_gait_params[trial_type].append(gait_params['spatio_temp'])
                        
                        for joint in part_kinematic_params[trial_type]:
                            for i, side in enumerate(part_kinematic_params[trial_type][joint]):
                                part_kinematic_params[trial_type][joint][i] = np.append(part_kinematic_params[trial_type][joint][i], gait_params['kinematics'][joint][i], axis=0) 

                    else:
                        part_strides[trial_type] = partitioned_mvn_data
                        part_kinematic_params[trial_type] = gait_params['kinematics']
                        part_gait_params[trial_type] = [gait_params['spatio_temp']]
                    
                    file_name = os.path.basename(blob.name)
                    
                    ##CHANGE THE excel reader - access the different 
                    XsensGaitParser_GN =  excel_reader_GN.XsensGaitDataParser()
                    XsensGaitParser_GN.process_mvn_trial_data(f"gs://{bucket_name}/{blob.name}")
                    partitioned_mvn_data_GN = XsensGaitParser_GN.get_partitioned_mvn_data()
                    gait_params_GN = XsensGaitParser_GN.get_gait_param_info()
                    
                    if trial_type in part_strides_INI:
                        for body_part in part_strides[trial_type]:
                            for i, side in enumerate(part_strides_INI[trial_type][body_part]):
                                # for each part (pelvis, l_hip, r_knee, etc.), append strides to appropriate list
                                part_strides_INI[trial_type][body_part][i] = part_strides_INI[trial_type][body_part][i] + partitioned_mvn_data_GN[body_part][i]
                        
                        for j, param in enumerate(part_gait_params_INI[trial_type]):
                            part_gait_params_INI[trial_type][j] = np.concatenate((part_gait_params_INI[trial_type][j], gait_params_GN['spatio_temp'][j]), axis=-1)

                    else:
                        part_strides_INI[trial_type] = partitioned_mvn_data_GN
                        part_gait_params_INI[trial_type] = gait_params_GN['spatio_temp']
                    

                except IndexError as e: #Exception based on an Index Error encountered in excel_reader_gcp.py **
                    #print(f"File skipped: gs://{bucket_name}/{blob.name} due to error: {e}")
                    continue                              
    
    if trial_type in part_gait_params:
        
        stance_time_symmetry = [item for sublist in [i[11] for i in part_gait_params[trial_type]] for item in sublist]
        
        partitioned_awinda_gait = {}
        partitioned_awinda_gait['pelvis_orient'] = reshape_vector(part_strides[trial_type]['pelvis_orient'][0], new_size = 51)
        partitioned_awinda_gait['hip_angle'] = [reshape_vector(part_strides[trial_type]['hip_angle'][0], new_size = 51), reshape_vector(part_strides[trial_type]['hip_angle'][1], new_size = 51)]
        partitioned_awinda_gait['knee_angle'] = [reshape_vector(part_strides[trial_type]['knee_angle'][0], new_size = 51), reshape_vector(part_strides[trial_type]['knee_angle'][1], new_size = 51)]
        partitioned_awinda_gait['ankle_angle'] = [reshape_vector(part_strides[trial_type]['ankle_angle'][0], new_size = 51), reshape_vector(part_strides[trial_type]['ankle_angle'][1], new_size = 51)]

        # Extract and reshape individual signals
        individual_signals = []
        gait_scores_list = []

        for i in range(partitioned_awinda_gait['pelvis_orient'].shape[0]):
            signal_dict = {
                'pelvis_orient': partitioned_awinda_gait['pelvis_orient'][i].reshape(1, 51, 3),
                'hip_angle': [partitioned_awinda_gait['hip_angle'][0][i].reshape(1, 51, 3), partitioned_awinda_gait['hip_angle'][1][i].reshape(1, 51, 3)],
                'knee_angle': [partitioned_awinda_gait['knee_angle'][0][i].reshape(1, 51, 3), partitioned_awinda_gait['knee_angle'][1][i].reshape(1, 51, 3)],
                'ankle_angle': [partitioned_awinda_gait['ankle_angle'][0][i].reshape(1, 51, 3), partitioned_awinda_gait['ankle_angle'][1][i].reshape(1, 51, 3)]
            }
            individual_signals.append(signal_dict)

        print(np.shape(individual_signals[0]['pelvis_orient']))    
        print(np.shape(individual_signals[0]['hip_angle'][0]))

        for signal_val in individual_signals:
            gait_scores = calc_gait_profile_score(signal_val, partitioned_awinda_control)
            gait_scores_list.append(gait_scores)
            
        flattened_raw_sensor = []
        part_strides_list = []
        for sublist in part_raw_sensor:
            for item in sublist:
                flattened_raw_sensor.append(item) #Flatten to individual gait cycles
                
        groups, gaitcycles, ordered_strides, ordered_params = check_group_configurations(stance_time_symmetry, flattened_raw_sensor, part_strides, part_gait_params_INI)
        
        #Establishing control as the first group
        participant_info = pd.read_csv(bucket_dir + 'Gait Quality Analysis/Data/Participant_Data/Raw Data/participant_info.csv')
        partitioned_gait_data = []
        
    
        for index, part_strides in enumerate(ordered_strides):
            
            params_INI = {
                'stride time': [],
                'stride length': [],
                'swing phase': [],
                'MAV': [],
                'MAH': [],
                'MHD': [],
                'MAB': [],
                'MAD': [],
                'ShROM': []
            }
            
            acc_scale = 0.02
            partitioned_signals_awinda_grouped = {
                'Pelvis_IMU': combine_acc_and_gyro(acc_scale * reshape_vector(part_strides['acc_data'][1]), reshape_vector(part_strides['gyro_data'][1])),
                'UpperR_IMU': combine_acc_and_gyro(acc_scale * reshape_vector(part_strides['acc_data'][2]), reshape_vector(part_strides['gyro_data'][2])),
                'UpperL_IMU': combine_acc_and_gyro(acc_scale * reshape_vector(part_strides['acc_data'][5]), reshape_vector(part_strides['gyro_data'][5])),
                'LowerR_IMU': combine_acc_and_gyro(acc_scale * reshape_vector(part_strides['acc_data'][3]), reshape_vector(part_strides['gyro_data'][3])),
                'LowerL_IMU': combine_acc_and_gyro(acc_scale * reshape_vector(part_strides['acc_data'][6]), reshape_vector(part_strides['gyro_data'][6])),
                'pelvis_orient': reshape_vector(part_strides['pelvis_orient'][0], new_size=51),
                'hip_angle': [reshape_vector(part_strides['hip_angle'][0], new_size=51), reshape_vector(part_strides['hip_angle'][1], new_size=51)],
                'knee_angle': [reshape_vector(part_strides['knee_angle'][0], new_size=51), reshape_vector(part_strides['knee_angle'][1], new_size=51)],
                'ankle_angle': [reshape_vector(part_strides['ankle_angle'][0], new_size=51), reshape_vector(part_strides['ankle_angle'][1], new_size=51)]
            }

            indiv_height = participant_info.loc[participant_info['Participant'] == participant]['Height (m)'].item()
            
            for i, param in enumerate(params_INI):
                params_INI[param] = ordered_params[index][i]
                if ini_height_norm[param]:
                    params_INI[param] = params_INI[param] / indiv_height

            # Check if the current index is 0 (control group)
            if index == 0:
                partitioned_control_data = AwindaData('control', dict(partitioned_signals_awinda_grouped), dict(params_INI))

            else:
                # Append to the regular partitioned_gait_data list
                partitioned_gait_data.append(AwindaData(trial_type, dict(partitioned_signals_awinda_grouped), dict(params_INI)))

        control_INI_params = []
        
        for param in params_INI:
            control_INI_params.append(partitioned_control_data.ini_gait_params[param][prosth_side])
        control_INI_params = np.transpose(np.array(control_INI_params))

        mean_control = np.mean(control_INI_params, axis = 0)
        std_control = np.std(control_INI_params, axis = 0)

        standardized_control = (control_INI_params - mean_control) / std_control
        cov_control = np.cov(standardized_control.T)
        
        eig_vals, eig_vecs = np.linalg.eig(cov_control)
        print(f"INI Score for participant: {participant}")
        for i in range(len(partitioned_gait_data)):
            side_1 = np.transpose(np.array( [partitioned_gait_data[i].ini_gait_params[param][0] for param in params_INI] ))
            side_2 = np.transpose(np.array( [partitioned_gait_data[i].ini_gait_params[param][1] for param in params_INI] ))
            test_ini = np.mean([side_1, side_2], axis=0)
            ini_data = calc_INI(mean_control, std_control, eig_vals, eig_vecs, test_ini)
            ini_score = np.mean(np.linalg.norm(ini_data, axis=-1))
            mean_STSR = np.mean(groups[i+1])
            add_row_to_csv(csv_path, 'STSR', 'INI', participant, mean_STSR, ini_score)
            print(mean_STSR)
            print(ini_score)    


<>:3: SyntaxWarning: invalid escape sequence '\G'
<>:3: SyntaxWarning: invalid escape sequence '\G'
C:\Users\ekuep\AppData\Local\Temp\ipykernel_18256\3323904910.py:3: SyntaxWarning: invalid escape sequence '\G'
  script_dir = 'C:\GP-WearablesAnalysis\examples'


Processing participant LLPU_P01
Prosthetic on the right side
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P01
6.50562763485156
8.511521288115723
6.896771175108751
8.634133148916126
7.321034235078354
8.766413808585563
Processing participant LLPU_P02
Prosthetic side on the left side
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P02
6.512184340920215
9.148542591102428
6.860426300371599
10.030791250716344
7.247605423502377
11.205134441382292
Processing participant LLPU_P03
Prosthetic side on the left side
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P03
5.359418524069248
7.4306921364707605
5.7234976421520765
7.987417266759122
6.108496371441923
8.676826546027538
Processing participant LLPU_P04
Prosthetic side on the left side
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P04
6.764157186620489
9.358730905030963
7.168498623522591
9.380516361518993
7.526307469256675
9.383592535372099
Processing participant LLPU_P05
Prosthetic side on the left side
(1, 51, 3)
(

c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\ekuep\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=

(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P08
6.710021000086634
9.090562539871886
7.070451533809153
9.861644064548972
7.440923546278977
10.634950739910655
Processing participant LLPU_P09
Prosthetic on the right side
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P09
4.638402509864576
5.948661227009609
4.997930291932548
5.892575865497267
5.38490319544396
5.9675267122596924
Processing participant LLPU_P10
Prosthetic on the right side
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P10
4.706182561868617
7.2091388830903815
5.072383239942215
7.3359189916764755
5.469521870929921
8.272248207180468
Processing participant LLPU_P12
Prosthetic on the right side
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P12
8.7564638553543
15.770494736231262
9.122605638590034
16.97069415556648
9.511039671929431
17.69451904896757
Processing participant LLPU_P15
Prosthetic on the right side
(1, 51, 3)
(1, 51, 3)
INI Score for participant: LLPU_P15
6.112090211498027
6.838128856963

In [ ]:
print(len(ordered_params))

for group in ordered_params:
    print(np.shape(group))

5
(9, 2, 100)
(9, 2, 50)
(9, 2, 50)
(9, 2, 50)
()
